In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Glaucoma_Detection')
sys.path.append('/content/drive/MyDrive/Glaucoma_Detection/utils')

In [2]:
pip install tensorboardx

     |████████████████████████████████| 122kB 6.5MB/s 


In [3]:
import torch.backends.cudnn as cudnn
import torch.optim as optim
from tqdm import tqdm
import random,sys,time
import os
import torch
from utils.extract_patches import get_data_train
from utils.losses.loss import *
from utils.visualize import group_images, save_img
from utils.common import *
from utils.dataset import TrainDataset,TestDataset
from torch.utils.data import DataLoader
# from config import parse_args
from utils.logger import Logger, Print_Logger
from collections import OrderedDict
from utils.metrics import Evaluate
from Model_Architecture import UNetFamily

In [4]:
#  Load the data and extract patches
def get_dataloader(train_data_path_list, train_patch_height, train_patch_width, stride_height, stride_width, val_ratio, batch_size, outf, save):
    patches_imgs_train, patches_masks_train = get_data_train(
        data_path_list = train_data_path_list,
        patch_height = train_patch_height,
        patch_width = train_patch_width,
        stride_height = stride_height,
        stride_width = stride_width
    )

    print(patches_imgs_train.shape, patches_masks_train.shape)

    val_ind = random.sample(range(patches_masks_train.shape[0]),int(np.floor(val_ratio*patches_masks_train.shape[0])))
    train_ind =  set(range(patches_masks_train.shape[0])) - set(val_ind)
    train_ind = list(train_ind)

    train_set = TrainDataset(patches_imgs_train[train_ind,...],patches_masks_train[train_ind,...],mode="train")
    train_loader = DataLoader(train_set, batch_size=batch_size,
                              shuffle=True, num_workers=4)

    val_set = TrainDataset(patches_imgs_train[val_ind,...],patches_masks_train[val_ind,...],mode="val")
    val_loader = DataLoader(val_set, batch_size=batch_size,
                            shuffle=False, num_workers=4)
    
    #Save some samples of feeding to the neural network
    # N_sample = min(patches_imgs_train.shape[0], 50)
    # save_img(group_images((patches_imgs_train[0:N_sample, :, :, :]*255).astype(np.uint8), 10),
    #           os.path.join(outf, save, "sample_input_imgs.png"))
    # save_img(group_images((patches_masks_train[0:N_sample, :, :, :]*255).astype(np.uint8), 10),
    #           os.path.join(outf, save,"sample_input_masks.png"))

    return train_loader,val_loader

# train 
def train(train_loader,net,criterion,optimizer,device):
    net.train()
    train_loss = AverageMeter()

    for batch_idx, (inputs, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss.update(loss.item(), inputs.size(0))
    log = OrderedDict([('train_loss',train_loss.avg)])
    return log

# val 
def val(val_loader,net,criterion,device):
    net.eval()
    val_loss = AverageMeter()
    evaluater = Evaluate()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in tqdm(enumerate(val_loader), total=len(val_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            val_loss.update(loss.item(), inputs.size(0))

            outputs = outputs.data.cpu().numpy()
            targets = targets.data.cpu().numpy()
            evaluater.add_batch(targets,outputs[:,1])
    log = OrderedDict([('val_loss', val_loss.avg), 
                       ('val_acc', evaluater.confusion_matrix()[1]), 
                       ('val_f1', evaluater.f1_score()),
                       ('val_auc_roc', evaluater.auc_roc())])
    return log

In [5]:
setpu_seed(2021)

outf = "/content/drive/MyDrive/Glaucoma_Detection/Experiments"
save = "Optic_Disc_Seg"
save_path = os.path.join(outf, save)

if not os.path.exists(save_path):
  os.mkdir(save_path)

device = torch.device("cuda" if torch.cuda.is_available() and True else "cpu")
cudnn.benchmark = True
log = Logger(save_path)
sys.stdout = Print_Logger(os.path.join(save_path,'train_log.txt'))
print('The computing device used is: ','GPU' if device.type=='cuda' else 'CPU')

net = UNetFamily.U_Net(1,2).to(device)
print("Total number of parameters: " + str(count_parameters(net)))
log.save_graph(net,torch.randn((1,1,128,128)).to(device).to(device=device))

N_epochs = 20
start_epoch = 1
criterion = CrossEntropyLoss2d() # Initialize loss function
optimizer = optim.Adam(net.parameters(), lr=0.0005)
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_epochs, eta_min=0)
train_data_path_list = "/content/drive/MyDrive/Glaucoma_Detection/Dataset_Preparation/data_path_list/IDRiD/train.txt"
train_patch_height, train_patch_width = 128, 128
stride_height, stride_width = 64, 64
val_ratio = 0.05
batch_size = 48

train_loader, val_loader = get_dataloader(train_data_path_list, train_patch_height, train_patch_width,
                                          stride_height, stride_width, val_ratio, batch_size, outf, save) # create dataloader

The computing device used is:  GPU
Total number of parameters: 34525954
Architecture of Model have saved in Tensorboard!
load data from /content/drive/MyDrive/Glaucoma_Detection/Dataset_Preparation/data_path_list/IDRiD/train.txt 


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


ori data shape < ori_imgs:(54, 3, 1112, 1012) GTs:(54, 1, 1112, 1012)
imgs pixel range 0-255: 
GTs pixel range 0-255: 
==================data have loaded======================

the side H is not compatible with the selected stride of 64
(img_h - patch_h) MOD stride_h: 24
So the H dim will be padded with additional 40 pixels
the side W is not compatible with the selected stride of 64
(img_w - patch_w) MOD stride_w: 52
So the W dim will be padded with additional 12 pixels
new padded images shape: (54, 1, 1152, 1024)

the side H is not compatible with the selected stride of 64
(img_h - patch_h) MOD stride_h: 24
So the H dim will be padded with additional 40 pixels
the side W is not compatible with the selected stride of 64
(img_w - patch_w) MOD stride_w: 52
So the W dim will be padded with additional 12 pixels
new padded images shape: (54, 1, 1152, 1024)

Train images shape: (54, 1, 1152, 1024), value range (0.0 - 1.0):

Traint masks shape: (54, 1, 1152, 1024), value range (0.0 - 1.0):
Nu

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
len(train_loader), len(val_loader)

(273, 15)

# Train

In [7]:
best = {'epoch':0,'AUC_roc':0.5} # Initialize the best epoch and performance(AUC of ROC)
trigger = 0  # Early stop Counter

for epoch in range(start_epoch,N_epochs+1):
  print('\nEPOCH: %d/%d --(learn_rate:%.6f) | Time: %s' % \
            (epoch, N_epochs,optimizer.state_dict()['param_groups'][0]['lr'], time.asctime()))
  
  # train stage
  train_log = train(train_loader,net,criterion, optimizer,device)
  # val stage
  val_log = val(val_loader,net,criterion,device)

  log.update(epoch,train_log,val_log) # Add log information
  lr_scheduler.step()

  # Save checkpoint of latest and best model
  state = {'net': net.state_dict(),'optimizer':optimizer.state_dict(),'epoch': epoch}
  # torch.save(state, os.path.join(save_path, 'latest_model.pth'))
  trigger += 1
  if val_log['val_auc_roc'] > best['AUC_roc']:
    print('\033[0;33mSaving best model!\033[0m')
    torch.save(state, os.path.join(save_path, 'best_model.pth'))
    best['epoch'] = epoch
    best['AUC_roc'] = val_log['val_auc_roc']
    trigger = 0

  print('Best performance at Epoch: {} | AUC_roc: {}'.format(best['epoch'],best['AUC_roc']))

  # early stopping
  early_stop = 6
  if not early_stop is None:
    if trigger >= early_stop:
      print("=> early stopping")
      break

  torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 15/15 [00:05<00:00,  2.76it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 15/15 [00:04<00:00,  3.

# Visualization

In [8]:
net.train()
for batch_idx, (inputs, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
  inputs, targets = inputs.to(device), targets.to(device)
  optimizer.zero_grad()
  outputs = net(inputs)
  break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/205 [00:04<?, ?it/s]


In [9]:
type(inputs), type(outputs), type(targets)

(torch.Tensor, torch.Tensor, torch.Tensor)

In [12]:
input = inputs.cpu().detach().numpy()
output = outputs.cpu().detach().numpy()
target = targets.cpu().detach().numpy()

type(input), type(output), type(target)

(numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [13]:
input.shape, output.shape, target.shape

((64, 1, 128, 128), (64, 2, 128, 128), (64, 128, 128))

In [ ]:
import matplotlib.pyplot as plt

for i in range(64):
  plt.imshow(input[i,0,:,:], cmap="gray")
  plt.show()
  plt.imshow(output[i,0,:,:], cmap="gray")
  plt.show()
  plt.imshow(target[i,:,:], cmap="gray")
  plt.show()